In [8]:
import os
import cv2
import csv
import numpy as np
from numpy import ndarray
from brisque import BRISQUE

In [9]:
os.environ['OPENCV_IO_ENABLE_OPENEXR'] = "1"

In [10]:
def read_exr(im_path: str) -> ndarray:
    return cv2.imread(filename=im_path, flags=cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)

def resize_image(image: ndarray, size: tuple = (256, 256)) -> ndarray:
    return cv2.resize(image, size, interpolation=cv2.INTER_LINEAR)

def tone_map_reinhard(image: ndarray) -> ndarray: 
    tonemap_operator = cv2.createTonemapReinhard(gamma=2.2, intensity=0.0, light_adapt=0.0, color_adapt=0.0)
    result = tonemap_operator.process(src=image)
    return result

def tone_map_mantiuk(image: ndarray) -> ndarray:
    tonemap_operator = cv2.createTonemapMantiuk(gamma=2.2, scale=0.85, saturation=1.2)
    result = tonemap_operator.process(src=image)
    return result

def evaluate_image(image: ndarray) -> float:
    image = (image * 255).clip(0, 255).astype(np.uint8)
    
    metric = BRISQUE(url=False)
    return metric.score(img=image)

def save_png(image: ndarray, output_path: str):
    image_8bit = (image * 255).clip(0, 255).astype(np.uint8) 
    cv2.imwrite(output_path, image_8bit)

def save_exr(image: ndarray, output_path: str):
    cv2.imwrite(output_path, image.astype(np.float32)) 

In [11]:
INPUT_DIR = "../data/reference/"
OUTPUT_DIR = "../data/output_dir"
REINHARD_DIR = "../data/reinhard_output/"
MANTIUK_DIR = "../data/mantiuk_output/"
METRIC_CSV = "../data/brisque_MANTIUK_REINHARD.csv"
resized_scale = (256,256)

In [12]:
csv_headers = ["Filename", "REINHARD", "MANTIUK"]
with open(METRIC_CSV, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(csv_headers)

In [13]:
os.makedirs(REINHARD_DIR, exist_ok=True)
os.makedirs(MANTIUK_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [14]:
for filename in os.listdir(INPUT_DIR):
    if filename.endswith(".exr"):
        input_path = os.path.join(INPUT_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, filename)

        image = read_exr(input_path)
        image_resized = resize_image(image, resized_scale)
        save_exr(image_resized, output_path)

        tone_mapped_reinhard = tone_map_reinhard(image_resized)
        tone_mapped_mantiuk = tone_map_mantiuk(image_resized)

        reinhard_score = evaluate_image(tone_mapped_reinhard)
        mantiuk_score = evaluate_image(tone_mapped_mantiuk)

        reinhard_path = os.path.join(REINHARD_DIR, filename.replace(".exr", ".png"))
        mantiuk_path = os.path.join(MANTIUK_DIR, filename.replace(".exr", ".png"))
        
        with open(METRIC_CSV, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([filename, reinhard_score, mantiuk_score]
        )
        save_png(tone_mapped_reinhard, reinhard_path)
        save_png(tone_mapped_mantiuk, mantiuk_path)


/tmp/ipykernel_19857/2170540200.py:18: RuntimeWarning: invalid value encountered in cast
  image = (image * 255).clip(0, 255).astype(np.uint8)
/tmp/ipykernel_19857/2170540200.py:24: RuntimeWarning: invalid value encountered in cast
  image_8bit = (image * 255).clip(0, 255).astype(np.uint8)
